In [1]:
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
token="здесь должен быть токен"

In [3]:
group_name="kommersant"
offset=0
count=100
all_posts=[]
all_comments=[]

In [4]:
offset=0
while offset<1700:
        req=requests.get('https://api.vk.com/method/wall.search',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'domain': group_name,
                   'query': 'инфляция',
                   'count': count,
                   'offset': offset
                   
               })
        res=req.json()['response']['items']
        offset+=100
        time.sleep(1)
        all_posts.extend(res)

In [5]:
res=req.json()['response']['items']
all_posts.extend(res)
df = pd.json_normalize(all_posts)

In [6]:
df=df[["id", "owner_id", "date", "text", "comments.count"]]
df.rename(columns={'date': 'post_time','text':'post_text'}, inplace=True)
df=df[df['post_time']>=1630454400]
df=df[df['comments.count']!=0]
df

,id,owner_id,post_time,post_text,comments.count
0,2843169,-23482909,1674503281,Участники криптовалютного рынка продолжают игр...,2
1,2842149,-23482909,1674467704,,8
4,2840070,-23482909,1674374564,,26
5,2838400,-23482909,1674244833,Ожидаемый в этом году рост мировой экономики н...,5
7,2837676,-23482909,1674208905,В конце прошлой недели Росстат обнародовал под...,3
...,...,...,...,...,...
1755,2049650,-23482909,1630891270,Продолжение повышенной мировой инфляции после ...,1
1756,2049603,-23482909,1630885271,Электросетевой госхолдинг «Россети» после мног...,2
1758,2047811,-23482909,1630570231,Президент России Владимир Путин регулярно обсу...,19
1759,2047712,-23482909,1630552447,Это большая ошибка: утверждение о росте зарпла...,2


In [7]:
owner_id=-23482909
posts_id=list(df['id'])

In [8]:
for post_id in posts_id:
    req=requests.get('https://api.vk.com/method/wall.getComments',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'owner_id': owner_id,
                   'post_id': post_id
               })
    time.sleep(1)
    res=req.json()['response']['items']
    all_comments.extend(res)

In [9]:
data = pd.json_normalize(all_comments)
data

,id,from_id,date,text,post_id,owner_id,parents_stack,thread.count,thread.items,thread.can_post,thread.show_reply_button,attachments,deleted
0,2843171,221235737,1674503405,Подкрепился хомяками,2843169.0,-23482909.0,[],0,[],True,True,NaN,NaN
1,2843211,3938809,1674506458,"""Участники криптовалютного рынка продолжают иг...",2843169.0,-23482909.0,[],0,[],True,True,NaN,NaN
2,2842155,235192661,1674467892,"То есть вы опросили россиян, и на основе их ли...",2842149.0,-23482909.0,[],0,[],True,True,NaN,NaN
3,2842162,12025189,1674468038,Танки это 20 век ...в 21 танками уже никто не ...,2842149.0,-23482909.0,[],6,[],True,True,"[{'type': 'photo', 'photo': {'album_id': -8, '...",NaN
4,2840076,0,1674374864,,NaN,NaN,[],6,[],True,True,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3923,2047723,273184221,1630553570,а то что цены на все в 1.5-2 раза выросли - эт...,2047712.0,-23482909.0,[],0,[],True,True,NaN,NaN
3924,2047730,235505037,1630555502,"Спрос вырос, зарплата нет.",2047712.0,-23482909.0,[],0,[],True,True,NaN,NaN
3925,2047731,355644233,1630555610,при такой щедрости управленцы должны перебежат...,2047712.0,-23482909.0,[],0,[],True,True,NaN,NaN
3926,2048414,298551544,1630647312,И спрос на специалистов вырос и зарплаты вырос...,2047712.0,-23482909.0,[],0,[],True,True,NaN,NaN


In [10]:
data=data[["post_id", "text", "date"]]
data.rename(columns={'date': 'comment_time', 'id': 'comment_id'}, inplace=True)
data=data.dropna()
data['post_id'] = data['post_id'].astype(int)
data

,post_id,text,comment_time
0,2843169,Подкрепился хомяками,1674503405
1,2843169,"""Участники криптовалютного рынка продолжают иг...",1674506458
2,2842149,"То есть вы опросили россиян, и на основе их ли...",1674467892
3,2842149,Танки это 20 век ...в 21 танками уже никто не ...,1674468038
5,2840070,Цены как обычно будут расти. Это байденовский ...,1674374933
...,...,...,...
3923,2047712,а то что цены на все в 1.5-2 раза выросли - эт...,1630553570
3924,2047712,"Спрос вырос, зарплата нет.",1630555502
3925,2047712,при такой щедрости управленцы должны перебежат...,1630555610
3926,2047712,И спрос на специалистов вырос и зарплаты вырос...,1630647312


In [11]:
df.to_csv('kommersant_posts.csv', index=False)

In [12]:
data.to_csv('kommersant_comments.csv', index=False)